In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Überprüfen, ob eine GPU verfügbar ist, ansonsten die CPU verwenden
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Verwende {device} als Gerät")

Verwende cuda als Gerät


In [2]:
import torch

if torch.cuda.is_available():
    print("CUDA ist verfügbar! 🎉")
    print(f"Anzahl der GPUs: {torch.cuda.device_count()}")
    print(f"Name der GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA ist nicht verfügbar. Überprüfe deine Treiber und die PyTorch-Installation.")

CUDA ist verfügbar! 🎉
Anzahl der GPUs: 1
Name der GPU: NVIDIA GeForce RTX 3060


In [3]:
# Trainingsdaten herunterladen
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(), # Umwandlung der Bilder in Tensoren
)

# Testdaten herunterladen
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# DataLoader erstellen
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Überprüfen der Datenform
for X, y in test_dataloader:
    print(f"Form von X [N, C, H, W]: {X.shape}") # N=Batch Size, C=Channels, H=Height, W=Width
    print(f"Form von y: {y.shape} {y.dtype}")
    break

Form von X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Form von y: torch.Size([64]) torch.int64


In [4]:
# Definiert eine neue Klasse namens "NeuralNetwork".
# Sie erbt von "nn.Module", der Basisklasse für alle neuronalen Netz-Module in PyTorch.
# Das bedeutet, unsere Klasse bekommt alle Funktionalitäten von PyTorch, z.B. das Verwalten von Parametern (Gewichten) und die Fähigkeit, auf einer GPU zu laufen.
class NeuralNetwork(nn.Module):
    
    # Der Konstruktor der Klasse. Diese Methode wird aufgerufen, wenn ein neues Objekt
    # dieser Klasse erstellt wird (z.B. mit model = NeuralNetwork()).
    # Hier werden die Bausteine (Layer) des Netzes definiert und initialisiert.
    def __init__(self):
        # Ruft den Konstruktor der übergeordneten Klasse (nn.Module) auf.
        # Dies ist ein notwendiger Schritt, um sicherzustellen, dass alles korrekt
        # initialisiert wird, damit PyTorch die Layer verwalten kann.
        super().__init__()
        
        # Erstellt eine "Flatten"-Schicht. Diese Schicht hat eine einfache Aufgabe:
        # Sie wandelt einen mehrdimensionalen Tensor in einen eindimensionalen Vektor um.
        # Ein Bild der Größe 28x28 Pixel wird also zu einem Vektor der Länge 784 (28*28).
        # Dies ist notwendig, um die Bilddaten in die nachfolgenden, vollvernetzten (Linear) Schichten zu geben.
        self.flatten = nn.Flatten()
        
        # "nn.Sequential" ist ein Container, der mehrere Schichten zu einem einzigen
        # Block zusammenfasst. Die Daten durchlaufen die Schichten in genau der Reihenfolge,
        # in der sie hier definiert sind. Das macht den Code im "forward"-Teil übersichtlicher.
        self.linear_relu_stack = nn.Sequential(
            # 1. Schicht (Input Layer): Eine vollvernetzte ("Linear") Schicht.
            # Sie erwartet einen Input mit 784 Features (vom geflatteten 28x28 Bild)
            # und transformiert diese linear (y = Wx + b) auf 128 Features.
            nn.Linear(28*28, 128),
            
            # 2. Schicht (Aktivierungsfunktion): Die ReLU (Rectified Linear Unit) Funktion.
            # Sie führt Nicht-Linearität in das Modell ein, was entscheidend ist,
            # um komplexe Muster zu lernen. Sie setzt alle negativen Werte auf 0 und lässt
            # positive Werte unverändert.
            nn.ReLU(),
            
            # 3. Schicht (Hidden Layer): Eine weitere vollvernetzte Schicht.
            # Sie nimmt die 128 Features von der vorherigen Schicht und gibt wieder 128 Features aus.
            nn.Linear(128, 128),
            
            # 4. Schicht (Aktivierungsfunktion): Erneut eine ReLU-Aktivierung.
            nn.ReLU(),
            
            # 5. Schicht (Output Layer): Die letzte vollvernetzte Schicht.
            # Sie nimmt die 128 Features der letzten versteckten Schicht und erzeugt 10 Ausgabewerte.
            # Diese 10 Werte werden als "Logits" bezeichnet und repräsentieren die Roh-Scores
            # für jede der 10 möglichen Klassen (z.B. Ziffern 0-9).
            nn.Linear(128, 10)     
        )

    # Die "forward"-Methode definiert den eigentlichen "Vorwärtsdurchlauf" der Daten.
    # Wenn Daten an das Modell übergeben werden (z.B. model(input_data)), wird diese Methode ausgeführt.
    # 'x' ist hier der Input-Tensor, der die Daten enthält (z.B. ein Batch von Bildern).
    def forward(self, x):
        # Zuerst wird der Input 'x' durch die Flatten-Schicht geschickt, um ihn zu einem Vektor zu machen.
        x = self.flatten(x)
        
        # Der geflattete Vektor 'x' wird dann durch den gesamten "linear_relu_stack" geleitet.
        # Das Ergebnis sind die finalen Logits.
        logits = self.linear_relu_stack(x)
        
        # Die Methode gibt die berechneten Logits zurück.
        # Diese können später mit einer Softmax-Funktion in Wahrscheinlichkeiten umgerechnet werden.
        return logits

# Erstellt eine Instanz unseres eben definierten neuronalen Netzes.
# ".to(device)" verschiebt das gesamte Modell, inklusive aller seiner Parameter (Gewichte und Biases),
# auf das angegebene Rechengerät 'device' (typischerweise 'cpu' oder 'cuda' für eine NVIDIA GPU).
# Die Variable 'device' muss zuvor definiert worden sein.
model = NeuralNetwork().to(device)

# Gibt eine textuelle Zusammenfassung der Modellarchitektur aus.
# Dies zeigt alle in `__init__` definierten Module und Schichten in ihrer Struktur an.
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [5]:
# Hier wird die Verlustfunktion (Loss Function) initialisiert.
# nn.CrossEntropyLoss ist eine sehr gebräuchliche Verlustfunktion für Klassifikationsprobleme mit mehreren Klassen.
# Unter der Haube kombiniert sie zwei Schritte:
# 1. Eine LogSoftmax-Funktion, die die rohen Logits des Modells in logarithmische Wahrscheinlichkeiten umwandelt.
# 2. Die Negative Log Likelihood Loss (NLLLoss), die misst, wie gut das Modell die korrekte Klasse vorhersagt.
# Die Kombination in einer Klasse ist numerisch stabiler und effizienter als die separate Ausführung.
# Das Ziel des Trainings wird es sein, den von dieser Funktion berechneten Wert zu minimieren.
loss_fn = nn.CrossEntropyLoss()

# Definiert die Lernrate (Learning Rate). Dies ist ein sogenannter Hyperparameter,
# der steuert, wie stark die Gewichte des Modells bei jedem Trainingsschritt angepasst werden.
# "1e-3" ist die wissenschaftliche Schreibweise für 0.001.
# Eine zu hohe Lernrate kann dazu führen, dass das Training instabil wird; eine zu niedrige
# Lernrate kann das Training extrem verlangsamen.
learning_rate = 1e-3

# Hier wird der Optimierungsalgorithmus (Optimizer) erstellt.
# Der Optimizer hat die Aufgabe, die Gewichte des Modells anzupassen, um den Verlust (loss) zu reduzieren.
# torch.optim.Adam: Wir wählen den Adam-Optimizer, einen sehr populären und oft effektiven
# Algorithmus. Adam passt die Lernrate für jeden einzelnen Parameter des Modells adaptiv an,
# was oft zu schnellerer Konvergenz führt als bei einfacheren Methoden wie SGD.
#
# model.parameters(): Dieser Aufruf übergibt dem Optimizer eine Liste aller lernbaren Parameter
# (also aller Gewichte und Biases) unseres Modells. Der Optimizer weiß somit, welche Werte er anpassen muss.
#
# lr=learning_rate: Hiermit wird die anfängliche Lernrate für den Optimizer auf den Wert gesetzt,
# den wir zuvor definiert haben.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# - dataloader: Ein PyTorch DataLoader, der die Trainingsdaten in Batches bereitstellt.
# - model: Das neuronale Netz, das trainiert werden soll.
# - loss_fn: Die Verlustfunktion, um den Fehler des Modells zu berechnen.
# - optimizer: Der Optimierungsalgorithmus, der die Gewichte des Modells anpasst.
def train(dataloader, model, loss_fn, optimizer):
    # Ermittelt die Gesamtgröße des Datensatzes für die Fortschrittsanzeige.
    size = len(dataloader.dataset)
    
    # Versetzt das Modell in den "Trainingsmodus".
    # Das ist wichtig, da einige Schichten (wie Dropout oder BatchNorm) sich im
    # Trainings- und im Auswertungsmodus unterschiedlich verhalten.
    model.train()
    
    # Die Hauptschleife, die über alle Daten-Batches des DataLoaders iteriert.
    # "enumerate" liefert zusätzlich einen Zähler ("batch") für den aktuellen Batch.
    # Jeder Batch enthält die Eingabedaten 'X' (Bilder) und die zugehörigen Labels 'y' (korrekte Ziffern).
    for batch, (X, y) in enumerate(dataloader):
        # Verschiebt die Daten des aktuellen Batches (Bilder und Labels) auf das
        # ausgewählte Rechengerät ('device', also CPU oder GPU).
        # Dies ist notwendig, damit die Berechnungen auf der GPU stattfinden können, falls verfügbar.
        X, y = X.to(device), y.to(device)

        # === Schritt 1: Forward Pass ===
        # Die Eingabedaten 'X' werden durch das Modell geschickt. Das Modell berechnet
        # für jeden Input die Vorhersagen (Logits).
        pred = model(X)
        
        # Die Verlustfunktion vergleicht die Vorhersagen des Modells ('pred') mit den
        # wahren Labels ('y') und berechnet den Fehler (Loss) für diesen Batch.
        loss = loss_fn(pred, y)

        # === Schritt 2: Backward Pass (Backpropagation) ===
        # PyTorch berechnet automatisch die Gradienten (Ableitungen) des Verlusts
        # in Bezug auf alle lernbaren Parameter (Gewichte) des Modells.
        # Diese Gradienten geben an, wie stark jeder Parameter zum Fehler beigetragen hat.
        loss.backward()
        
        # === Schritt 3: Optimierung ===
        # Der Optimizer aktualisiert die Gewichte des Modells. Er verwendet die im
        # .backward()-Schritt berechneten Gradienten und seine Update-Regel (z.B. Adam),
        # um die Gewichte leicht zu verändern und so den Verlust zu minimieren.
        optimizer.step()
        
        # Die berechneten Gradienten müssen nach jedem Update-Schritt zurückgesetzt werden.
        # PyTorch akkumuliert Gradienten standardmäßig. Ohne .zero_grad() würden sich
        # die Gradienten von Batch zu Batch aufsummieren, was zu falschen Updates führen würde.
        optimizer.zero_grad()

        # Dieser Block dient nur zur Ausgabe des Fortschritts alle 100 Batches.
        if batch % 100 == 0:
            # .item() extrahiert den reinen Python-Zahlenwert aus dem Loss-Tensor.
            loss_value = loss.item()
            # Berechnet, wie viele Datenpunkte bisher in dieser Epoche verarbeitet wurden.
            current = (batch + 1) * len(X)
            # Gibt den aktuellen Verlust und den Fortschritt aus.
            print(f"Verlust: {loss_value:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
# Definiert die Funktion zur Evaluierung des Modells auf einem Test- oder Validierungsdatensatz.
# Sie benötigt den entsprechenden DataLoader, das Modell und die Verlustfunktion.
def test(dataloader, model, loss_fn):
    # Ermittelt die Gesamtgröße des Datensatzes und die Anzahl der Batches.
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    # Versetzt das Modell in den "Evaluierungsmodus".
    # Dies ist das Gegenstück zu model.train() und stellt sicher, dass Schichten
    # wie Dropout deaktiviert werden, um konsistente Ergebnisse zu erhalten.
    model.eval()
    
    # Initialisiert Variablen, um den Gesamtverlust und die Anzahl der korrekten
    # Vorhersagen über alle Batches hinweg zu summieren.
    test_loss, correct = 0, 0
    
    # "torch.no_grad()" ist ein Kontextmanager, der die Berechnung von Gradienten deaktiviert.
    # Da wir das Modell hier nur testen und nicht trainieren, brauchen wir keine Gradienten.
    # Das spart Speicher und beschleunigt die Berechnungen erheblich.
    with torch.no_grad():
        # Schleife, die über alle Batches des Test-DataLoaders iteriert.
        for X, y in dataloader:
            # Verschiebt die Daten des aktuellen Batches auf das Rechengerät (CPU/GPU).
            X, y = X.to(device), y.to(device)
            
            # === Forward Pass ===
            # Die Eingabedaten 'X' werden durch das Modell geschickt, um die Vorhersagen 'pred' zu erhalten.
            pred = model(X)
            
            # Berechnet den Verlust für den aktuellen Batch und addiert ihn zum Gesamtverlust.
            # .item() extrahiert den reinen Python-Zahlenwert aus dem Tensor.
            test_loss += loss_fn(pred, y).item()
            
            # Zählt die Anzahl der korrekten Vorhersagen in diesem Batch:
            # 1. pred.argmax(1): Findet für jede Eingabe den Index (die Klasse) mit dem höchsten Vorhersagewert.
            # 2. == y: Vergleicht die vorhergesagten Klassen mit den wahren Labels 'y'. Das Ergebnis ist ein Tensor mit True/False-Werten.
            # 3. .type(torch.float): Wandelt den True/False-Tensor in einen Tensor mit 1.0/0.0-Werten um.
            # 4. .sum(): Summiert alle Einsen auf, um die Anzahl der korrekten Vorhersagen zu erhalten.
            # 5. .item(): Extrahiert die Summe als Python-Zahl und addiert sie zur Gesamtsumme.
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    # Berechnet den durchschnittlichen Verlust pro Batch.
    test_loss /= num_batches
    
    # Berechnet die Genauigkeit (Accuracy), indem die Gesamtzahl der korrekten Vorhersagen
    # durch die Gesamtgröße des Datensatzes geteilt wird.
    correct /= size
    
    # Gibt eine formatierte Zusammenfassung der Ergebnisse aus.
    print(f"Test Fehler: \n Genauigkeit: {(100*correct):>0.1f}%, Avg Verlust: {test_loss:>8f} \n")

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoche {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Training abgeschlossen!")

Epoche 1
-------------------------------
Verlust: 2.303267  [   64/60000]
Verlust: 0.339238  [ 6464/60000]
Verlust: 0.230606  [12864/60000]
Verlust: 0.268099  [19264/60000]
Verlust: 0.144082  [25664/60000]
Verlust: 0.290033  [32064/60000]
Verlust: 0.141095  [38464/60000]
Verlust: 0.274460  [44864/60000]
Verlust: 0.279492  [51264/60000]
Verlust: 0.224199  [57664/60000]
Test Fehler: 
 Genauigkeit: 94.6%, Avg Verlust: 0.168203 

Epoche 2
-------------------------------
Verlust: 0.121027  [   64/60000]
Verlust: 0.098320  [ 6464/60000]
Verlust: 0.103754  [12864/60000]
Verlust: 0.097643  [19264/60000]
Verlust: 0.053172  [25664/60000]
Verlust: 0.141850  [32064/60000]
Verlust: 0.059070  [38464/60000]
Verlust: 0.191124  [44864/60000]
Verlust: 0.184738  [51264/60000]
Verlust: 0.147611  [57664/60000]
Test Fehler: 
 Genauigkeit: 96.8%, Avg Verlust: 0.105084 

Epoche 3
-------------------------------
Verlust: 0.051860  [   64/60000]
Verlust: 0.058553  [ 6464/60000]
Verlust: 0.080086  [12864/60000]


In [9]:
# Modell wieder in den Eval-Modus versetzen
model.eval()

# Ein einzelnes Bild aus dem Testdatensatz nehmen
x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    x = x.to(device)
    logits = model(x)
    # Manuell Softmax anwenden, um Wahrscheinlichkeiten zu erhalten
    probabilities = nn.functional.softmax(logits, dim=1)
    prediction = probabilities.argmax(1)
    
    print(f"Echtes Label: {y}")
    print(f"Vorhergesagtes Label: {prediction.item()}")
    print(f"Wahrscheinlichkeiten pro Klasse:\n {probabilities.cpu().numpy().flatten()}")

Echtes Label: 7
Vorhergesagtes Label: 7
Wahrscheinlichkeiten pro Klasse:
 [1.4358498e-09 1.8572498e-08 4.1451617e-08 3.2412430e-08 3.7418841e-09
 6.9738125e-11 2.5839524e-15 9.9999845e-01 3.8575190e-10 1.5233356e-06]


In [10]:
# Beispiel für eine modifizierte Netzwerkarchitektur (nur zur Veranschaulichung)
# Sie würden dies in Zelle 3 einfügen und diese erneut ausführen.
class ModifiedNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 256), # Geändert auf 256 Neuronen
            nn.ReLU(),
            nn.Linear(256, 256),   # Geändert auf 256 Neuronen
            nn.ReLU(),
            nn.Linear(256, 128),   # Neue dritte versteckte Schicht
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# model = ModifiedNeuralNetwork().to(device) # So würden Sie das neue Modell initialisieren